In [29]:
def google_trends(Country, Searches_list, your_path):
    
    global df_trends, Searches, Folder_path
    
    from datetime import datetime, date
    import pytrends
    import pandas as pd
    from pytrends.request import TrendReq
    pytrend = TrendReq()

    Folder_path = your_path
    Searches = Searches_list #["bitcoin", "maker", "solana", "aave", "fantom", "dash crypto"]
    Searches_codes = [pytrend.suggestions(keyword=i)[0] for i in Searches]
    df = pd.DataFrame(Searches_codes)

    Actual_Keywords=df['mid'].to_list()
    today = date.today().isoformat()
    begin_date = datetime.now().date().replace(month=1, day=1)
    Date_of_interest= str(begin_date) + ' ' + today
    Countries_of_interest = Country
    Category=0 
    Search_type='' #default is 'web searches',others include 'images','news','youtube','froogle' (google shopping)

    Individual_Actual_Keywords = list(zip(*[iter(Actual_Keywords)]*1))
    Individual_Actual_Keywords = [list(x) for x in Individual_Actual_Keywords]
    dict_it = {}
    i = 1
    for Country in Countries_of_interest:
        for keyword in Individual_Actual_Keywords:
            pytrend.build_payload(kw_list=keyword, 
                                  timeframe = Date_of_interest, 
                                  geo = Country, 
                                  cat=Category,
                                  gprop=Search_type) 
            dict_it[i] = pytrend.interest_over_time()
            i+=1
    df_trends = pd.concat(dict_it, axis=1)

    df_trends.columns = df_trends.columns.droplevel(0) #drop outside header
    df_trends = df_trends.drop('isPartial', axis = 1) #drop "isPartial"
    df_trends.reset_index(level=0,inplace=True) #reset_index
    new_list = ['date']
    df_trends.columns= new_list + Searches#['date', Searches[0], Searches[1],Searches[2], Searches[3], Searches[4], Searches[5]]#'aave', 'maker', 'solana', 'compound', 'fantom', 'dash crypto']
    df_trends.to_csv(today + ".csv")
    return

def plot_trends(i):
    import plotly.express as px

    fig = px.line(data_frame=df_trends, x='date', y=Searches[i], labels={
                         "date": "Date",
                         Searches[i]: Searches[i].capitalize()
                     },
                    title="Google Trend for " + Searches[i].capitalize())
    path = Folder_path + "/" + Searches[i] + ".html"
    fig.write_html(path)
    return fig.show()

def sendemail(extension:str):
    
    import os
    import imghdr
    import smtplib
    from email.message import EmailMessage
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    
    server.login('bamideleemmanuel02@gmail.com', 'bnngusejgmsqdhrh')
    
    newMessage = EmailMessage() 
    newMessage['Subject'] = "Google Trends for selected cryptocurrencies"
    newMessage['From'] = 'bamideleemmanuel02@gmail.com'
    recipients = ['218045808@student.uj.ac.za', 'hello@emmanuelbamidele.com']
    newMessage['To'] = ", ".join(recipients)
    newMessage.set_content('Attached are the trends for selected cryptocurrencies')
    
    files = [img for img in os.listdir(Folder_path) if img.endswith(extension)]
    for file in files:
        with open(file, 'rb') as f:
            image_data = f.read()
            image_type = imghdr.what(f.name)
            image_name = f.name
        newMessage.add_attachment(image_data, maintype='image', subtype="html", filename=image_name)
    
    server.send_message(newMessage)
    print('Hey Email sent')
    
    server.quit

In [31]:
google_trends(Country= ["US"], Searches_list=["bitcoin", "eth", "solana", "aave", "litecoin"], your_path = "/Users/emmanuel_bamidele/Desktop/Bamspace/Data_Science_Tutorial/Machine Learning/Google Trends")

for coin in range(len(Searches)):
    plot_trends(coin)
    
sendemail(extension = "html")

SMTPSenderRefused: (552, b"5.2.3 Your message exceeded Google's message size limits. Please visit\n5.2.3  https://support.google.com/mail/?p=MaxSizeError to view our size\n5.2.3 guidelines. e17sm1222679iow.18 - gsmtp", 'bamideleemmanuel02@gmail.com')